1. 결측치 찾기

In [10]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import math

df = pd.read_csv('../data/raw/Customer_Churn_Dataset.csv')

In [ ]:
for i in df.columns:
    print("column name :",i, ", \t sum of empty column:", df[i].isin(['', ' ']).sum())

In [ ]:
df[df['TotalCharges'].isin(['', ' '])]

In [ ]:
print(df['TotalCharges'].isin(['', ' ']).sum())
print(df['TotalCharges'].isin(['']).sum())
print(df['TotalCharges'].isin([' ']).sum())

In [ ]:
df[df['TotalCharges'].isin([' '])]

In [ ]:
pattern2 = r"^\s*$" # 공백만 있는 문자열
df['TotalCharges'].str.contains(pattern2, regex=True).sum()

2. 결측치 대체

2-1. 형변환

In [ ]:
print(df.dtypes)

In [11]:
df_missing = df.copy(deep=False)

TotalCharges를 형변환 하려고 했으나, 에러

In [ ]:
# df2['TotalCharges'] = df2['TotalCharges'].astype(np.float64)
df_missing['TotalCharges'] = pd.to_numeric(df_missing['TotalCharges'])

빈 문자열 때문에 에러 발생. 형변환하면서 결측치는 'NaN'으로 표기하도록 처리

In [12]:
df_missing['TotalCharges'] = pd.to_numeric(df_missing['TotalCharges'], errors='coerce')

In [ ]:
df_missing[df_missing['TotalCharges'].isnull()]

In [ ]:
df_missing.isnull().sum()

In [ ]:
df_missing.dtypes

2-2. 결측치 대체

2-2-1. SimpleImputer 이용

2-2-1-1. 중앙값 대체

In [13]:
df_missing_simple1 = df_missing.copy(deep=False)

In [14]:
from sklearn.impute import SimpleImputer
# SimpleImputer로 중앙값
imputer1 = SimpleImputer(strategy="median")
imputer1.fit(df_missing_simple1[['TotalCharges']])  # 결측치를 어떤 값으로 바꿀지 학습. (2차원 -> 0축 기준으로 계산)
result1 = imputer1.transform(df_missing_simple1[['TotalCharges']])  # 변환작업 (fit에서 찾은 중앙값으로 결측치를 대체)

In [15]:
df_missing_simple1['TotalCharges'] = result1

In [16]:
df_missing_simple1.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
numAdminTickets     0
numTechTickets      0
Churn               0
dtype: int64

In [17]:
df_missing_simple1.to_csv('../data/raw/Customer_Churn_Dataset_simple_median.csv')

2-2-1-2. 최빈값 대체

In [18]:
df_missing_simple2 = df_missing.copy(deep=False)

In [19]:
from sklearn.impute import SimpleImputer
imputer2 = SimpleImputer(strategy="most_frequent")

# imputer1.fit(df_missing[['TotalCharges']])  # 결측치를 어떤 값으로 바꿀지 학습. (2차원 -> 0축 기준으로 계산)
# result1 = imputer1.transform(df[['A', 'B']])  # 변환작업 (fit에서 찾은 중앙값으로 결측치를 대체)
result2 = imputer2.fit_transform(df_missing_simple2['TotalCharges'].to_frame()) #series.to_frame() : Series->DataFrame

In [20]:
df_missing_simple2['TotalCharges'] = result2

In [21]:
df_missing_simple2.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
numAdminTickets     0
numTechTickets      0
Churn               0
dtype: int64

In [22]:
df_missing_simple2.to_csv('../data/raw/Customer_Churn_Dataset_simple_frequent.csv')

2-2-2. KNN 이용

In [28]:
df_missing_knn = df_missing.copy(deep=False)

In [29]:
from sklearn.impute import KNNImputer
# 2. KNNImputer 객체 생성 및 설정
# n_neighbors 파라미터로 사용할 이웃의 수를 지정합니다 (기본값은 5)
imputer = KNNImputer(n_neighbors=5) # 예시로 2개의 이웃 사용

# 3. 데이터에 imputer 적용 (fit_transform)
# 결측값이 채워진 numpy 배열 반환
imputed_data = imputer.fit_transform(df_missing_knn['TotalCharges'].to_frame())

df_missing_knn['TotalCharges'] = imputed_data

In [30]:
df_missing_knn.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
numAdminTickets     0
numTechTickets      0
Churn               0
dtype: int64

In [31]:
df_missing_knn.to_csv('../data/raw/Customer_Churn_Dataset_knn.csv')